In [1]:
# import
import pandas as pd
import numpy as np
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew, kurtosis
import utils

import NN
import os
from torch import load as load_model
import config as cfg

In [2]:
LSTM_RESULTS_PATH = "models_norm/LSTM_preds"
MLP_RESULTS_PATH = "models_norm/MLP_preds"
RNN_RESULTS_PATH = "models_norm/RNN_preds"
PSN_RESULTS_PATH = "models_norm/PSN_preds"

In [18]:
# all data
dia_df = pd.read_csv("data/DIA.csv")
qqq_df = pd.read_csv("data/QQQ.csv")
spy_df = pd.read_csv("data/SPY.csv")

# risk free
riskfree = pd.read_csv("data/Tbill/Valid.csv")
riskfreeDf = riskfree[['Date', 'Adj Close']].copy()
riskfreeDf['Date'] = pd.to_datetime(riskfreeDf['Date'])
riskfreeDf.set_index('Date', inplace=True)
riskfreeDf.rename(columns={'Adj Close':'Risk_free'}, inplace=True)
riskfreeRate = riskfreeDf["Risk_free"].mean()

# adding suffix to all df
dia_df.columns=dia_df.columns.map(lambda x : x+'_dia' if x !='Date' else x)
qqq_df.columns=qqq_df.columns.map(lambda x : x+'_qqq' if x !='Date' else x)
spy_df.columns=spy_df.columns.map(lambda x : x+'_spy' if x !='Date' else x)

dfs = [dia_df, qqq_df, spy_df]

df = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)

# set date column as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# keeping only the correct date 03/01/2011 to 13/04/2015
Total_df = df.loc[('2011-01-03' <= df.index) & (df.index <= '2015-04-13')]
Training_df = df.loc[('2011-01-03' <= df.index) & (df.index <= '2012-12-31')]
Test_df = df.loc[('2013-01-02' <= df.index) & (df.index <= '2013-12-31')]
Out_of_sample_df = df.loc[('2014-01-02' <= df.index) & (df.index <= '2015-04-13')]

# ptf portfolio
Total_adj_close_df = Total_df[['Adj Close_dia', 'Adj Close_qqq', 'Adj Close_spy']].copy()
Total_adj_close_df[['prev_adj_Close_dia', 'prev_adj_Close_qqq', 'prev_adj_Close_spy']] = Total_adj_close_df[['Adj Close_dia', 'Adj Close_qqq', 'Adj Close_spy']].shift(periods=1)

# weeklyDf = Total_adj_close_df.resample('D').interpolate()[::7]
weeklyDf = Total_adj_close_df.copy()

weeklyDf['DIA'] = np.log(weeklyDf['Adj Close_dia']) - np.log(weeklyDf['prev_adj_Close_dia'])
weeklyDf['QQQ'] = np.log(weeklyDf['Adj Close_qqq']) - np.log(weeklyDf['prev_adj_Close_qqq'])
weeklyDf['SPY'] = np.log(weeklyDf['Adj Close_spy']) - np.log(weeklyDf['prev_adj_Close_spy'])


if not os.path.isdir("portfolio"):
    os.mkdir("portfolio")

df = weeklyDf[['DIA', 'QQQ', 'SPY']].copy()
df.to_csv("portfolio/ETF_returns.csv", index=True)

# keeping only Out-of-Sample
weeklyDf = weeklyDf[(weeklyDf.index >= '2014-01-01')]
weeklyDf = weeklyDf[['DIA', 'QQQ', 'SPY']]
weeklyDf['1/N'] = (1/3) * (weeklyDf['DIA'] + weeklyDf['QQQ'] + weeklyDf['SPY'])

In [19]:
# keeping only Out-of-Sample
weeklyDf

,DIA,QQQ,SPY,1/N
Date,,,,
2014-01-02,-0.008253,-0.007876,-0.009630,-0.008586
2014-01-03,0.001705,-0.007245,-0.000164,-0.001902
2014-01-06,-0.003046,-0.003700,-0.002902,-0.003216
2014-01-07,0.006568,0.009225,0.006123,0.007305
2014-01-08,-0.003765,0.002179,0.000218,-0.000456
...,...,...,...,...
2015-04-07,-0.000952,-0.001700,-0.002650,-0.001767
2015-04-08,0.001848,0.007908,0.003371,0.004376
2015-04-09,0.003629,0.006263,0.004414,0.004769


# Trading perf

In [20]:
Table7 = pd.DataFrame({})

for ptf in ['SPY', 'DIA', 'QQQ', '1/N']:    
    weeklyDf[ ptf + '_Cumulative_return' ] = (1 + weeklyDf[ ptf ]).cumprod()
    
    annualizedReturn = weeklyDf[ ptf ].cumsum().to_list()[-1] * 252 * (1/len(weeklyDf))
    annualizedVol = weeklyDf[ ptf ].std() * np.sqrt(252)
    downsideVol = weeklyDf.loc[weeklyDf[ ptf ] < 0][ ptf ].std() * np.sqrt(252)
    maximumDrawdown = 100 * utils.MDD(weeklyDf, ptf + "_Cumulative_return", len(weeklyDf))
    sharpeRatio = (100 * annualizedReturn - riskfreeRate)/ (100 * annualizedVol)
    sortinoRatio = (100 * annualizedReturn - riskfreeRate)/ (100 * downsideVol)

    Table = pd.DataFrame({'Strategies':[ptf],
                          'Realized return (%)':[100 * annualizedReturn],
                          'Sharpe Ratio':[sharpeRatio],
                          'Sortino Ratio':[sortinoRatio],
                          'Max drawdown (%)':[maximumDrawdown]
                         })    
    Table7 = pd.concat([Table7,Table],ignore_index=True)
Table7.set_index(["Strategies"], inplace=True)

In [21]:
Table7

,Realized return (%),Sharpe Ratio,Sortino Ratio,Max drawdown (%)
Strategies,,,,
SPY,11.610182,0.785876,1.087209,-7.375937
DIA,8.543654,0.531960,0.739849,-6.931405
QQQ,17.108159,1.053719,1.420190,-8.384762
1/N,12.420665,0.833935,1.142806,-7.456279


# Variance Prediction

## Preparing the data to do it on Matlab

In [22]:
# generating the table to do that on Matlab
df = df[(df.index >= '2014-01-01')]
df

,DIA,QQQ,SPY
Date,,,
2014-01-02,-0.008253,-0.007876,-0.009630
2014-01-03,0.001705,-0.007245,-0.000164
2014-01-06,-0.003046,-0.003700,-0.002902
2014-01-07,0.006568,0.009225,0.006123
2014-01-08,-0.003765,0.002179,0.000218
...,...,...,...
2015-04-07,-0.000952,-0.001700,-0.002650
2015-04-08,0.001848,0.007908,0.003371
2015-04-09,0.003629,0.006263,0.004414


In [23]:
for ETF in ['DIA', 'SPY', 'QQQ']:
    
    df.reset_index(inplace=True)
    for NNtype in ['MLP', 'RNN', 'PSN', 'LSTM', 'ARMA']:
        if NNtype != 'ARMA':            
            RESULTS_PATH = "models_norm/" + NNtype + "_preds"
            NNpreds = pd.read_csv(RESULTS_PATH + "/Predictions")[ETF]

            # adding the NN predictions
            df[ ETF + "_" + NNtype + "_Pred" ] = NNpreds

        else:
            df.set_index(["Date"], inplace=True)

            # Loading ARMA prediction for this ETF
            ARMApredsDf = pd.read_pickle(os.path.join("models_norm", ETF, "ARMA", "Predictions.pkl"))
            df = df.join(ARMApredsDf)
            df.rename(columns={'Predictions': ETF + '_ARMA_Pred'}, inplace=True)
riskfree = pd.read_csv("data/Tbill/Valid.csv")
rf = riskfree[['Date', 'Adj Close']].copy()
rf['Date'] = pd.to_datetime(rf['Date'])
rf.set_index('Date', inplace=True)
rf.rename(columns={'Adj Close':'Risk_free'}, inplace=True)

# adding Risk Free rate
df = df.join(rf)

# adding all the data
dfbis = pd.read_csv("portfolio/ETF_returns.csv")
dfbis['Date'] = pd.to_datetime(dfbis['Date'])
dfbis.set_index('Date', inplace=True)
df = dfbis.merge(df, left_index=True, right_index=True, how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
df

,DIA,QQQ,SPY,DIA_MLP_Pred,DIA_RNN_Pred,DIA_PSN_Pred,DIA_LSTM_Pred,DIA_ARMA_Pred,SPY_MLP_Pred,SPY_RNN_Pred,SPY_PSN_Pred,SPY_LSTM_Pred,SPY_ARMA_Pred,QQQ_MLP_Pred,QQQ_RNN_Pred,QQQ_PSN_Pred,QQQ_LSTM_Pred,QQQ_ARMA_Pred,Risk_free
Date,,,,,,,,,,,,,,,,,,,
2014-01-02,-0.008253,-0.007876,-0.009630,0.006527,0.008368,0.005599,0.007920,0.000442,0.002878,0.002069,0.007012,0.009402,0.000525,-0.000072,0.001727,-0.001625,0.006045,0.000776,2.985
2014-01-03,0.001705,-0.007245,-0.000164,-0.003022,-0.000687,0.007442,-0.001088,0.001039,-0.001845,-0.003665,0.013391,-0.001351,0.000957,-0.003595,0.002616,-0.001121,0.005138,0.000326,2.995
2014-01-06,-0.003046,-0.003700,-0.002902,0.000266,0.001469,0.009405,-0.003293,-0.000147,0.001003,0.000665,0.017525,0.003241,0.000402,-0.009907,0.003140,-0.001466,0.000588,0.000471,2.961
2014-01-07,0.006568,0.009225,0.006123,0.001150,0.005131,0.013931,-0.000065,0.000541,-0.002400,0.003031,0.021145,0.006240,0.000782,-0.002541,-0.001104,0.000306,-0.005636,0.000769,2.937
2014-01-08,-0.003765,0.002179,0.000218,0.000395,-0.003396,0.005757,-0.002302,0.000754,-0.001003,0.000897,0.013731,0.003339,0.001288,0.000727,0.001042,-0.000222,0.010150,0.001629,2.993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-07,-0.000952,-0.001700,-0.002650,0.001114,0.002715,0.004234,-0.004788,-0.000221,-0.000034,-0.006843,0.011963,-0.007877,0.000355,0.002161,-0.000126,0.003843,0.000260,0.002190,1.893
2015-04-08,0.001848,0.007908,0.003371,0.004605,0.002135,0.021713,0.003896,0.003289,0.002601,0.002216,0.021728,-0.002109,0.000082,-0.003172,-0.001781,0.006232,0.016447,0.001130,1.895
2015-04-09,0.003629,0.006263,0.004414,-0.005286,-0.007493,0.022959,0.000724,-0.001226,0.001317,-0.002582,0.011477,0.007004,0.000299,0.005154,0.013716,0.002501,0.005940,0.001109,1.958


In [40]:
df

,DIA,QQQ,SPY,DIA_MLP_Pred,DIA_RNN_Pred,DIA_PSN_Pred,DIA_LSTM_Pred,DIA_ARMA_Pred,SPY_MLP_Pred,SPY_RNN_Pred,SPY_PSN_Pred,SPY_LSTM_Pred,SPY_ARMA_Pred,QQQ_MLP_Pred,QQQ_RNN_Pred,QQQ_PSN_Pred,QQQ_LSTM_Pred,QQQ_ARMA_Pred,Risk_free
Date,,,,,,,,,,,,,,,,,,,
2011-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,0.001973,-0.000724,-0.000551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,0.003424,0.008468,0.005184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-06,-0.002225,0.003224,-0.001960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-07,-0.001800,-0.000895,-0.001964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-07,-0.000952,-0.001700,-0.002650,0.001114,0.002715,0.004234,-0.004788,-0.000221,-0.000034,-0.006843,0.011963,-0.007877,0.000355,0.002161,-0.000126,0.003843,0.000260,0.002190,1.893
2015-04-08,0.001848,0.007908,0.003371,0.004605,0.002135,0.021713,0.003896,0.003289,0.002601,0.002216,0.021728,-0.002109,0.000082,-0.003172,-0.001781,0.006232,0.016447,0.001130,1.895
2015-04-09,0.003629,0.006263,0.004414,-0.005286,-0.007493,0.022959,0.000724,-0.001226,0.001317,-0.002582,0.011477,0.007004,0.000299,0.005154,0.013716,0.002501,0.005940,0.001109,1.958


In [37]:
dfbis.index

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14',
               ...
               '2015-03-30', '2015-03-31', '2015-04-01', '2015-04-02',
               '2015-04-06', '2015-04-07', '2015-04-08', '2015-04-09',
               '2015-04-10', '2015-04-13'],
              dtype='datetime64[ns]', name='Date', length=1075, freq=None)

In [56]:
for NNtype in ['MLP', 'RNN', 'PSN', 'LSTM']:
    col = ['DIA_' + NNtype + '_Pred', 'SPY_' + NNtype + '_Pred', 'QQQ_' + NNtype + '_Pred', 'Risk_free']
    NNdf = df[col].copy()
    
    if not os.path.isdir("portfolio"):
        os.mkdir("portfolio")
    
    if not os.path.isdir("portfolio/" + NNtype):
        os.mkdir("portfolio/" + NNtype)
    
    FILE_NAME = "portfolio/" + NNtype + "/" + NNtype + "_optimal_ptf_data.csv"
    NNdf.to_csv(FILE_NAME, index=True)

In [57]:
NNdf

,DIA_LSTM_Pred,SPY_LSTM_Pred,QQQ_LSTM_Pred,Risk_free
Date,,,,
2014-01-02,0.007920,0.009402,0.006045,2.985
2014-01-03,-0.001088,-0.001351,0.005138,2.995
2014-01-06,-0.003293,0.003241,0.000588,2.961
2014-01-07,-0.000065,0.006240,-0.005636,2.937
2014-01-08,-0.002302,0.003339,0.010150,2.993
...,...,...,...,...
2015-04-07,-0.004788,-0.007877,0.000260,1.893
2015-04-08,0.003896,-0.002109,0.016447,1.895
2015-04-09,0.000724,0.007004,0.005940,1.958


# To WORK

In [7]:
# DCC GARCH
# DCC-GARCH(1,1)
# https://pypi.org/project/mgarch/

In [8]:
import mgarch

In [9]:
df[(df.index >= '2013-12-01')]

,DIA,QQQ,SPY
Date,,,
2013-12-02,-0.004991,-0.002921,-0.002600
2013-12-03,-0.005832,-0.000819,-0.004330
2013-12-04,-0.001448,0.000936,-0.000111
2013-12-05,-0.004039,-0.001287,-0.004405
2013-12-06,0.012819,0.007236,0.011115
...,...,...,...
2015-04-07,-0.000952,-0.001700,-0.002650
2015-04-08,0.001848,0.007908,0.003371
2015-04-09,0.003629,0.006263,0.004414


In [10]:
# rt = (t, n) numpy matrix with t days of observation and n number of assets
vol = mgarch.mgarch()
vol.fit(df[(df.index >= '2015-04-10')].to_numpy())
ndays = 1 # volatility of nth day
cov_nextday = vol.predict(ndays)

C:\Users\frede\Anaconda3\lib\site-packages\mgarch\mgarch.py:65: RuntimeWarning: invalid value encountered in log
  np.log(np.linalg.det(R_t[i])) + \


In [11]:
df[(df.index >= '2015-04-05')][['DIA', 'SPY']]

,DIA,SPY
Date,,
2015-04-06,0.007080,0.006710
2015-04-07,-0.000952,-0.002650
2015-04-08,0.001848,0.003371
2015-04-09,0.003629,0.004414
2015-04-10,0.005393,0.005442
2015-04-13,-0.004055,-0.004533


In [12]:
df[(df.index >= '2015-04-05')][['DIA', 'SPY']].cov()

,DIA,SPY
DIA,0.000017,0.000018
SPY,0.000018,0.000021


In [13]:
vol.fit(df[(df.index >= '2015-04-05')][['DIA', 'SPY']].to_numpy())
ndays = 7 # volatility of nth day
cov_nextday = vol.predict(ndays)

In [14]:
cov_nextday

{'dist': 'norm',
 'cov': array([[1.61130618e-05, 1.86005942e-05],
        [1.86005942e-05, 2.24510585e-05]])}